# Klasifikasi Berita menggunakan Machine Learning

Notebook ini berisi implementasi klasifikasi berita berbahasa Indonesia menggunakan beberapa algoritma machine learning. Proses yang dilakukan meliputi preprocessing teks, vectorization, dan evaluasi model.

## Library yang Digunakan

1. **pandas**: Untuk manipulasi dan analisis data dalam bentuk DataFrame
2. **re (Regular Expression)**: Untuk pembersihan teks menggunakan pattern matching
3. **nltk**: Natural Language Processing Toolkit untuk pemrosesan bahasa alami
   - stopwords: Untuk menghilangkan kata-kata umum yang tidak memiliki makna penting
   - word_tokenize: Untuk memecah teks menjadi token-token kata
4. **Sastrawi**: Library stemming untuk Bahasa Indonesia
5. **sklearn**: Library utama untuk machine learning
   - TfidfVectorizer: Untuk mengubah teks menjadi representasi numerik
   - Algoritma klasifikasi: KNN, SVM, dan Random Forest
   - Metrics: Untuk evaluasi performa model

## Struktur Project
```
project/
├── data/
│   └── berita.csv
├── model/
│   ├── tfidf_vectorizer.pkl
│   ├── knn_model.pkl
│   ├── svm_model.pkl
│   └── rf_model.pkl
├── 1_scraping_berita.ipynb
└── 2_klasifikasi_berita.ipynb
```

In [1]:
# Import library yang dibutuhkan
import pandas as pd
import re
import os
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Membuat direktori model jika belum ada
if not os.path.exists('model'):
    os.makedirs('model')

## 1. Persiapan Data

Membaca dataset berita dan melakukan pembersihan data awal.

In [2]:
# Membaca dataset
data = pd.read_csv('data/berita.csv')

# Membersihkan data
data = data.dropna(subset=['judul', 'konten', 'kategori'])  # Menghapus baris dengan nilai kosong
data = data.drop_duplicates()  # Menghapus duplikat

# Menampilkan informasi dataset
print("Informasi Dataset:")
print(f"Jumlah data: {len(data)}")
print("\nDistribusi kategori:")
print(data['kategori'].value_counts())

Informasi Dataset:
Jumlah data: 852

Distribusi kategori:
kategori
food        100
properti    100
sport        98
oto          97
finance      96
inet         91
travel       90
health       90
edu          90
Name: count, dtype: int64


## 2. Preprocessing Data

Melakukan preprocessing teks meliputi:
1. Pembersihan teks (cleaning)
2. Case folding (lowercase)
3. Tokenisasi
4. Penghapusan stopwords
5. Stemming

In [3]:
# Filter kategori yang valid
valid_kategori = ['finance', 'inet', 'sport', 'oto', 'travel', 'food', 'health', 'edu', 'properti']
data = data[data['kategori'].isin(valid_kategori)]

def clean_text(text):
    """Membersihkan teks dari karakter yang tidak diperlukan"""
    text = text.lower()  # Mengubah ke lowercase
    text = re.sub(r'\d+', '', text)  # Menghapus angka
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus tanda baca
    text = re.sub(r'\s+', ' ', text).strip()  # Menghapus whitespace berlebih
    return text

# Terapkan pembersihan teks
data['judul'] = data['judul'].apply(clean_text)
data['konten'] = data['konten'].apply(clean_text)

In [4]:
# Inisialisasi stopwords dan stemmer
stop_words = set(stopwords.words('indonesian'))
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

def preprocess_text(text):
    """Melakukan tokenisasi, penghapusan stopwords, dan stemming"""
    tokens = word_tokenize(text)  # Tokenisasi
    tokens = [word for word in tokens if word not in stop_words]  # Hapus stopwords
    stemmed_tokens = [stemmer.stem(word) for word in tokens]  # Stemming
    return ' '.join(stemmed_tokens)

# Terapkan preprocessing
data['judul'] = data['judul'].apply(preprocess_text)
data['konten'] = data['konten'].apply(preprocess_text)

# Gabungkan judul dan konten
data['text'] = data['judul'] + " " + data['konten']
X = data['text']
y = data['kategori']

## 3. Vectorization

Mengubah teks menjadi representasi numerik menggunakan TF-IDF (Term Frequency-Inverse Document Frequency)

In [5]:
# Vectorisasi menggunakan TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Membatasi fitur menjadi 5000 kata teratas
X = vectorizer.fit_transform(X)

# Simpan vectorizer
with open('model/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## 4. Training Model

Melatih tiga model klasifikasi berbeda:
1. K-Nearest Neighbors (KNN)
2. Support Vector Machine (SVM)
3. Random Forest

In [6]:
# Training dan menyimpan model KNN
print("Training model KNN...")
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
with open('model/knn_model.pkl', 'wb') as f:
    pickle.dump(knn_model, f)

# Training dan menyimpan model SVM
print("Training model SVM...")
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)
with open('model/svm_model.pkl', 'wb') as f:
    pickle.dump(svm_model, f)

# Training dan menyimpan model Random Forest
print("Training model Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
with open('model/rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

Training model KNN...
Training model SVM...
Training model Random Forest...


## 5. Evaluasi Model

Mengevaluasi performa setiap model menggunakan metrics:
- Precision
- Recall
- F1-score
- Accuracy

In [7]:
def evaluate_model(model, X_test, y_test, model_name):
    """Fungsi untuk mengevaluasi model dan menampilkan hasil evaluasi"""
    print(f"Evaluasi Model {model_name}:")
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    print("-" * 60)

# Evaluasi semua model
models = [
    (knn_model, "KNN"),
    (svm_model, "SVM"),
    (rf_model, "Random Forest")
]

for model, name in models:
    evaluate_model(model, X_test, y_test, name)

Evaluasi Model KNN:
              precision    recall  f1-score   support

         edu       0.75      0.67      0.71        18
     finance       0.62      0.68      0.65        19
        food       0.89      0.85      0.87        20
      health       0.80      0.89      0.84        18
        inet       0.80      0.89      0.84        18
         oto       0.83      0.75      0.79        20
    properti       0.82      0.90      0.86        20
       sport       0.90      0.95      0.93        20
      travel       1.00      0.78      0.88        18

    accuracy                           0.82       171
   macro avg       0.82      0.82      0.82       171
weighted avg       0.83      0.82      0.82       171

------------------------------------------------------------
Evaluasi Model SVM:
              precision    recall  f1-score   support

         edu       0.87      0.72      0.79        18
     finance       0.79      0.79      0.79        19
        food       0.95      0.

## 6. Contoh Penggunaan Model

Berikut adalah contoh cara menggunakan model yang telah disimpan untuk melakukan prediksi pada teks baru.

In [8]:
def load_models():
    """Fungsi untuk memuat model yang telah disimpan"""
    with open('model/vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)
    with open('model/rf_model.pkl', 'rb') as f:
        model = pickle.load(f)
    return vectorizer, model

def predict_category(text, vectorizer, model):
    """Fungsi untuk memprediksi kategori dari teks berita baru"""
    # Preprocessing teks
    cleaned_text = clean_text(text)
    processed_text = preprocess_text(cleaned_text)
    
    # Vectorisasi
    text_vector = vectorizer.transform([processed_text])
    
    # Prediksi
    prediction = model.predict(text_vector)[0]
    return prediction

# Contoh penggunaan
vectorizer, model = load_models()
contoh_teks = """Harga saham teknologi mengalami kenaikan signifikan"""

prediksi = predict_category(contoh_teks, vectorizer, model)
print(f"Kategori prediksi: {prediksi}")

Kategori prediksi: finance
